In [118]:
import warnings
warnings.filterwarnings('ignore')

import ast
from catboost import CatBoostRegressor
from collections import defaultdict, Counter
from concurrent.futures import ThreadPoolExecutor
from lightgbm import LGBMRegressor
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from openfe import OpenFE, transform, tree_to_formula, formula_to_tree, get_candidate_features, TwoStageFeatureSelector, ForwardFeatureSelector
import optuna
import os

import pandas as pd
from pprint import pprint

import random

from sklearn.ensemble import ExtraTreesRegressor, HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression, RFECV
from sklearn.inspection import permutation_importance
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler

from scipy import stats

import time
from tqdm.notebook import tqdm

from xgboost import XGBRegressor

pd.set_option('display.max_columns', None)

experiment_name = 'openfe'

In [2]:
# Load the train and test datasets
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train.shape, df_test.shape

((15289, 18), (10194, 17))

In [3]:
binary_cols = []

cat_cols = ['clonesize', 'honeybee', 'bumbles', 'andrena', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays', 'AverageRainingDays']

num_cols = ['fruitset', 'fruitmass', 'seeds']

ordinal_cols = []

In [4]:
TARGET = 'yield'

In [5]:
# Separate features and target for train and validation data

X_train = df_train.drop(['id', TARGET], axis=1)
y_train = df_train[TARGET]

X_test = df_test.drop(['id'], axis=1)

k10 = KFold(n_splits=3, random_state=5, shuffle=True)

In [6]:
X_train.head()

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds
0,25.0,0.50,0.25,0.75,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.425011,0.417545,32.460887
1,25.0,0.50,0.25,0.50,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.444908,0.422051,33.858317
2,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.552927,0.470853,38.341781
3,12.5,0.25,0.25,0.63,0.50,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.565976,0.478137,39.467561
4,25.0,0.50,0.25,0.63,0.63,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.579677,0.494165,40.484512


In [7]:
X_test.head()

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds
0,25.0,0.25,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.399367,0.408088,31.394569
1,12.5,0.25,0.25,0.75,0.63,94.6,57.2,79.0,68.2,33.0,55.9,1.0,0.10,0.488048,0.442866,36.846956
2,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.583379,0.487057,40.037644
3,25.0,0.50,0.38,0.38,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.433014,0.422847,33.116091
4,37.5,0.75,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,24.0,0.39,0.360996,0.388860,29.558019


In [8]:
# Define pipelines
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=50))
])

ridge_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nystroem', Nystroem(n_components=500, random_state=5)),
    ('ridge', Ridge())
])

linear_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', LinearRegression()),
])

# Manually set pipeline names
knn_pipeline.name = 'KNN'
ridge_pipeline.name = 'Nystroem Ridge'
linear_pipeline.name = 'LR Pipeline'

In [9]:
models = [
    CatBoostRegressor(random_state=5, verbose=False, early_stopping_rounds=100),
    ExtraTreesRegressor(random_state=5),
    # GaussianNB(),
    HistGradientBoostingRegressor(random_state=5),
    knn_pipeline,
    linear_pipeline,
    LGBMRegressor(n_jobs=-1, random_state=5),
    LinearRegression(),
    RandomForestRegressor(random_state=5),
    XGBRegressor(random_state=5),
]

In [10]:
def evaluate_models(models, X, y, important_features, cv_split, experiment_name):
    Model_compare = pd.DataFrame(columns=['Model Name', 
                                        'Model Parameters', 
                                        'Model Train Accuracy', 
                                        'Model Test Accuracy', 
                                        'Model Test Accuracy Std', 
                                        'Model Time'])
    
    def evaluate_model(alg, idx):
        if hasattr(alg, 'name'):
            model_name = alg.name
        else:
            model_name = alg.__class__.__name__
        features = important_features.get(model_name, [])

        # Check if the list of important features is empty
        if len(features) == 0:
            # If empty, return results with zero values
            print(f'Skipping {model_name} due to no important features.')
            return {
                'Model Name': model_name,
                'Model Parameters': str(alg.get_params()),
                'Model Train Accuracy': 0,
                'Model Test Accuracy': 0,
                'Model Test Accuracy Std': 0,
                'Model Time': "0 min 0.00 sec",
            }
        
        cv_results = cross_validate(alg, 
                                    X[features], 
                                    y, cv=cv_split, 
                                    scoring='neg_mean_absolute_error', 
                                    return_train_score=True, 
                                    n_jobs=-1)

        # Time formatting
        mean_fit_time = cv_results['fit_time'].mean()
        minutes, seconds = divmod(mean_fit_time, 60)

        # Results population
        result = {
            'Model Name': model_name,
            'Model Parameters': str(alg.get_params()),
            'Model Train Accuracy': -cv_results['train_score'].mean(),
            'Model Test Accuracy': -cv_results['test_score'].mean(),
            'Model Test Accuracy Std': cv_results['test_score'].std(),
            'Model Time': f"{int(minutes)} min {seconds:.2f} sec",
        }

        print(f'Done with {model_name}.')
        return result

    results_list = []

    with ThreadPoolExecutor(max_workers=50) as executor:
        futures = [executor.submit(evaluate_model, alg, idx) for idx, alg in enumerate(tqdm(models, desc='Models'))]
        for future in tqdm(futures, total=len(futures), desc='Progress'):
            result = future.result()
            results_list.append(result)

    model_compare = pd.DataFrame(results_list)

    model_compare.sort_values(by=['Model Test Accuracy'], ascending=True, inplace=True)
    model_compare.to_csv(f'{experiment_name}_results.csv', index=False)

    return model_compare

In [11]:
baseline_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    baseline_features[model_name] = list(X_train.columns)

In [12]:
%%time

baseline_models = evaluate_models(models, X_train, y_train, baseline_features, k10, f'{experiment_name}')
baseline_models

Models:   0%|          | 0/9 [00:00<?, ?it/s]

Progress:   0%|          | 0/9 [00:00<?, ?it/s]

Done with HistGradientBoostingRegressor.
Done with LGBMRegressor.
Done with XGBRegressor.
Done with KNN.
Done with LinearRegression.
Done with LR Pipeline.
Done with ExtraTreesRegressor.
Done with CatBoostRegressor.
Done with RandomForestRegressor.
CPU times: total: 312 ms
Wall time: 27 s


,Model Name,Model Parameters,Model Train Accuracy,Model Test Accuracy,Model Test Accuracy Std,Model Time
2,HistGradientBoostingRegressor,"{'categorical_features': None, 'early_stopping...",328.666081,355.479528,1.452854,0 min 0.94 sec
5,LGBMRegressor,"{'boosting_type': 'gbdt', 'class_weight': None...",314.487103,356.119295,0.761421,0 min 0.50 sec
0,CatBoostRegressor,"{'loss_function': 'RMSE', 'verbose': False, 'r...",300.162681,358.121005,0.588185,0 min 11.08 sec
7,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",137.575689,367.968505,0.945830,0 min 10.00 sec
4,LR Pipeline,"{'memory': None, 'steps': [('scaler', Standard...",370.792226,371.855852,3.770318,0 min 0.05 sec
6,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",370.792226,371.855852,3.770318,0 min 0.02 sec
8,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",240.757336,372.808122,2.311827,0 min 3.48 sec
1,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.088081,381.150267,1.702694,0 min 10.78 sec
3,KNN,"{'memory': None, 'steps': [('scaler', Standard...",442.486157,452.155949,4.004162,0 min 0.03 sec


In [20]:
with open('baseline_features.txt', mode='w') as f:
    pprint(baseline_features, stream=f)

# OPENFE

In [21]:
candidate_features_list = get_candidate_features(numerical_features=num_cols,
                                                     categorical_features=cat_cols,
                                                     ordinal_features=[])

In [27]:
%%time

# stage2_metric in ['gain_importance', 'permutation']
# stage1_metric in ['predictive', 'corr', 'mi']
# metric in ['binary_logloss', 'multi_logloss', 'auc', 'rmse']

ofe = OpenFE()

features = ofe.fit(
    data=X_train,
    label=y_train,
    candidate_features_list=candidate_features_list,
    categorical_features=cat_cols,
    n_data_blocks=2,
    # feature_boosting=True,
    task='regression',
    # stage2_metric='permutation',
    metric='rmse', 
    n_jobs=4,
    seed=5,
)

# 1 minute

The number of candidate features is 534
Start stage I selection.


100%|██████████| 16/16 [00:20<00:00,  1.30s/it]


225 same features have been deleted.
Meet early-stopping in successive feature-wise halving.


100%|██████████| 16/16 [00:21<00:00,  1.36s/it]


The number of remaining candidate features is 304
Start stage II selection.


100%|██████████| 16/16 [00:19<00:00,  1.23s/it]


Finish data processing.
CPU times: total: 3.5 s
Wall time: 1min 6s


In [28]:
# Get the list of OpenFE features

new_features_list = [feature for feature in features]
len(new_features_list)

304

In [29]:
# Check the best 25 features with their real feature names

topk=25
print(F'The top {topk} generated features are:')
for feature in new_features_list[:topk]:
    print(tree_to_formula(feature))

The top 25 generated features are:
(fruitset*seeds)
residual(fruitset)
(fruitset+seeds)
GroupByThenRank(seeds,MaxOfUpperTRange)
round(seeds)
(fruitmass+seeds)
max(fruitmass,seeds)
GroupByThenRank(fruitset,AverageOfLowerTRange)
GroupByThenRank(fruitset,MinOfUpperTRange)
GroupByThenRank(fruitset,MaxOfUpperTRange)
(fruitset*fruitmass)
(fruitset-fruitmass)
GroupByThenRank(seeds,AverageOfLowerTRange)
GroupByThenRank(seeds,MaxOfLowerTRange)
(fruitmass/seeds)
GroupByThenRank(fruitset,bumbles)
GroupByThenRank(fruitset,honeybee)
residual(seeds)
freq(seeds)
(fruitmass*seeds)
GroupByThenRank(fruitmass,MinOfLowerTRange)
GroupByThenRank(seeds,bumbles)
(fruitset/fruitmass)
(fruitset/seeds)
freq(fruitset)


In [57]:
# Transform the train and validation data to have the engineered features

train_ofe, test_ofe = transform(X_train, X_test, new_features_list, n_jobs=4)

# 1 min

In [58]:
train_ofe = train_ofe.reset_index(drop=True)
test_ofe = test_ofe.reset_index(drop=True)
train_ofe.shape, test_ofe.shape

((15289, 320), (10194, 320))

In [59]:
# See if there are any missing rows

train_ofe[train_ofe.isna().any(axis=1)]

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,autoFE_f_0,autoFE_f_1,autoFE_f_2,autoFE_f_3,autoFE_f_4,autoFE_f_5,autoFE_f_6,autoFE_f_7,autoFE_f_8,autoFE_f_9,autoFE_f_10,autoFE_f_11,autoFE_f_12,autoFE_f_13,autoFE_f_14,autoFE_f_15,autoFE_f_16,autoFE_f_17,autoFE_f_18,autoFE_f_19,autoFE_f_20,autoFE_f_21,autoFE_f_22,autoFE_f_23,autoFE_f_24,autoFE_f_25,autoFE_f_26,autoFE_f_27,autoFE_f_28,autoFE_f_29,autoFE_f_30,autoFE_f_31,autoFE_f_32,autoFE_f_33,autoFE_f_34,autoFE_f_35,autoFE_f_36,autoFE_f_37,autoFE_f_38,autoFE_f_39,autoFE_f_40,autoFE_f_41,autoFE_f_42,autoFE_f_43,autoFE_f_44,autoFE_f_45,autoFE_f_46,autoFE_f_47,autoFE_f_48,autoFE_f_49,autoFE_f_50,autoFE_f_51,autoFE_f_52,autoFE_f_53,autoFE_f_54,autoFE_f_55,autoFE_f_56,autoFE_f_57,autoFE_f_58,autoFE_f_59,autoFE_f_60,autoFE_f_61,autoFE_f_62,autoFE_f_63,autoFE_f_64,autoFE_f_65,autoFE_f_66,autoFE_f_67,autoFE_f_68,autoFE_f_69,autoFE_f_70,autoFE_f_71,autoFE_f_72,autoFE_f_73,autoFE_f_74,autoFE_f_75,autoFE_f_76,autoFE_f_77,autoFE_f_78,autoFE_f_79,autoFE_f_80,autoFE_f_81,autoFE_f_82,autoFE_f_83,autoFE_f_84,autoFE_f_85,autoFE_f_86,autoFE_f_87,autoFE_f_88,autoFE_f_89,autoFE_f_90,autoFE_f_91,autoFE_f_92,autoFE_f_93,autoFE_f_94,autoFE_f_95,autoFE_f_96,autoFE_f_97,autoFE_f_98,autoFE_f_99,autoFE_f_100,autoFE_f_101,autoFE_f_102,autoFE_f_103,autoFE_f_104,autoFE_f_105,autoFE_f_106,autoFE_f_107,autoFE_f_108,autoFE_f_109,autoFE_f_110,autoFE_f_111,autoFE_f_112,autoFE_f_113,autoFE_f_114,autoFE_f_115,autoFE_f_116,autoFE_f_117,autoFE_f_118,autoFE_f_119,autoFE_f_120,autoFE_f_121,autoFE_f_122,autoFE_f_123,autoFE_f_124,autoFE_f_125,autoFE_f_126,autoFE_f_127,autoFE_f_128,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137,autoFE_f_138,autoFE_f_139,autoFE_f_140,autoFE_f_141,autoFE_f_142,autoFE_f_143,autoFE_f_144,autoFE_f_145,autoFE_f_146,autoFE_f_147,autoFE_f_148,autoFE_f_149,autoFE_f_150,autoFE_f_151,autoFE_f_152,autoFE_f_153,autoFE_f_154,autoFE_f_155,autoFE_f_156,autoFE_f_157,autoFE_f_158,autoFE_f_159,autoFE_f_160,autoFE_f_161,autoFE_f_162,autoFE_f_163,autoFE_f_164,autoFE_f_165,autoFE_f_166,autoFE_f_167,autoFE_f_168,autoFE_f_169,autoFE_f_170,autoFE_f_171,autoFE_f_172,autoFE_f_173,autoFE_f_174,autoFE_f_175,autoFE_f_176,autoFE_f_177,autoFE_f_178,autoFE_f_179,autoFE_f_180,autoFE_f_181,autoFE_f_182,autoFE_f_183,autoFE_f_184,autoFE_f_185,autoFE_f_186,autoFE_f_187,autoFE_f_188,autoFE_f_189,autoFE_f_190,autoFE_f_191,autoFE_f_192,autoFE_f_193,autoFE_f_194,autoFE_f_195,autoFE_f_196,autoFE_f_197,autoFE_f_198,autoFE_f_199,autoFE_f_200,autoFE_f_201,autoFE_f_202,autoFE_f_203,autoFE_f_204,autoFE_f_205,autoFE_f_206,autoFE_f_207,autoFE_f_208,autoFE_f_209,autoFE_f_210,autoFE_f_211,autoFE_f_212,autoFE_f_213,autoFE_f_214,autoFE_f_215,autoFE_f_216,autoFE_f_217,autoFE_f_218,autoFE_f_219,autoFE_f_220,autoFE_f_221,autoFE_f_222,autoFE_f_223,autoFE_f_224,autoFE_f_225,autoFE_f_226,autoFE_f_227,autoFE_f_228,autoFE_f_229,autoFE_f_230,autoFE_f_231,autoFE_f_232,autoFE_f_233,autoFE_f_234,autoFE_f_235,autoFE_f_236,autoFE_f_237,autoFE_f_238,autoFE_f_239,autoFE_f_240,autoFE_f_241,autoFE_f_242,autoFE_f_243,autoFE_f_244,autoFE_f_245,autoFE_f_246,autoFE_f_247,autoFE_f_248,autoFE_f_249,autoFE_f_250,autoFE_f_251,autoFE_f_252,autoFE_f_253,autoFE_f_254,autoFE_f_255,autoFE_f_256,autoFE_f_257,autoFE_f_258,autoFE_f_259,autoFE_f_260,autoFE_f_261,autoFE_f_262,autoFE_f_263,autoFE_f_264,autoFE_f_265,autoFE_f_266,autoFE_f_267,autoFE_f_268,autoFE_f_269,autoFE_f_270,autoFE_f_271,autoFE_f_272,autoFE_f_273,autoFE_f_274,autoFE_f_275,autoFE_f_276,autoFE_f_277,autoFE_f_278,autoFE_f_279,autoFE_f_280,autoFE_f_281,autoFE_f_282,autoFE_f_283,autoFE_f_284,autoFE_f_285,autoFE_f_286,autoFE_f_287,autoFE_f_288,autoFE_f_289,autoFE_f_290,autoFE_f_291,autoFE_f_292,autoFE_f_293,autoFE_f_294,autoFE_f_295,autoFE_f_296,autoFE_f_297,autoFE_f_298,autoFE_f_299,au

In [77]:
train_ofe['autoFE_f_45'].sort_values().unique()

[0.0, 1.0, 2.0, 3.0, 4.0, ..., 21.0, 22.0, 23.0, 24.0, 25.0]
Length: 26
Categories (26, float64): [0.0, 1.0, 2.0, 3.0, ..., 22.0, 23.0, 24.0, 25.0]

In [60]:
# Check what columns have missing NaN
# if the previous step is zero then nothing will show here

nan_cols = train_ofe.columns[train_ofe.isna().any()].tolist()
nan_cols

['autoFE_f_93',
 'autoFE_f_101',
 'autoFE_f_117',
 'autoFE_f_120',
 'autoFE_f_126',
 'autoFE_f_146',
 'autoFE_f_152',
 'autoFE_f_165',
 'autoFE_f_222',
 'autoFE_f_223']

In [61]:
# Get the list of OpenFE features but with their real names and add the original features
# I just prefer the DataFrame to have real feature names than a bunch of AutoFE_d_1

ofe_cols = [tree_to_formula(feature) for feature in new_features_list]
ofe_cols

['(fruitset*seeds)',
 'residual(fruitset)',
 '(fruitset+seeds)',
 'GroupByThenRank(seeds,MaxOfUpperTRange)',
 'round(seeds)',
 '(fruitmass+seeds)',
 'max(fruitmass,seeds)',
 'GroupByThenRank(fruitset,AverageOfLowerTRange)',
 'GroupByThenRank(fruitset,MinOfUpperTRange)',
 'GroupByThenRank(fruitset,MaxOfUpperTRange)',
 '(fruitset*fruitmass)',
 '(fruitset-fruitmass)',
 'GroupByThenRank(seeds,AverageOfLowerTRange)',
 'GroupByThenRank(seeds,MaxOfLowerTRange)',
 '(fruitmass/seeds)',
 'GroupByThenRank(fruitset,bumbles)',
 'GroupByThenRank(fruitset,honeybee)',
 'residual(seeds)',
 'freq(seeds)',
 '(fruitmass*seeds)',
 'GroupByThenRank(fruitmass,MinOfLowerTRange)',
 'GroupByThenRank(seeds,bumbles)',
 '(fruitset/fruitmass)',
 '(fruitset/seeds)',
 'freq(fruitset)',
 '(fruitmass-seeds)',
 'GroupByThenRank(seeds,RainingDays)',
 'GroupByThenRank(fruitmass,RainingDays)',
 'freq(fruitmass)',
 '(fruitset-seeds)',
 'max(fruitset,fruitmass)',
 'CombineThenFreq(bumbles,RainingDays)',
 'GroupByThenRank(fru

In [62]:
# Add the OpenFE features to the original features

original_feats = list(X_train.columns)
original_feats.extend(ofe_cols)
original_feats

['clonesize',
 'honeybee',
 'bumbles',
 'andrena',
 'osmia',
 'MaxOfUpperTRange',
 'MinOfUpperTRange',
 'AverageOfUpperTRange',
 'MaxOfLowerTRange',
 'MinOfLowerTRange',
 'AverageOfLowerTRange',
 'RainingDays',
 'AverageRainingDays',
 'fruitset',
 'fruitmass',
 'seeds',
 '(fruitset*seeds)',
 'residual(fruitset)',
 '(fruitset+seeds)',
 'GroupByThenRank(seeds,MaxOfUpperTRange)',
 'round(seeds)',
 '(fruitmass+seeds)',
 'max(fruitmass,seeds)',
 'GroupByThenRank(fruitset,AverageOfLowerTRange)',
 'GroupByThenRank(fruitset,MinOfUpperTRange)',
 'GroupByThenRank(fruitset,MaxOfUpperTRange)',
 '(fruitset*fruitmass)',
 '(fruitset-fruitmass)',
 'GroupByThenRank(seeds,AverageOfLowerTRange)',
 'GroupByThenRank(seeds,MaxOfLowerTRange)',
 '(fruitmass/seeds)',
 'GroupByThenRank(fruitset,bumbles)',
 'GroupByThenRank(fruitset,honeybee)',
 'residual(seeds)',
 'freq(seeds)',
 '(fruitmass*seeds)',
 'GroupByThenRank(fruitmass,MinOfLowerTRange)',
 'GroupByThenRank(seeds,bumbles)',
 '(fruitset/fruitmass)',
 '(f

In [63]:
# Do the feature name change to understandable names
# Check the names of the columns with NaN

known_names_df = train_ofe.copy()

if len(original_feats) == len(train_ofe.columns):
    known_names_df.columns = original_feats
    print('OFE column names have changed to understandable column names')

known_col_names_nan_cols = known_names_df.columns[known_names_df.isna().any()].tolist()
known_col_names_nan_cols

OFE column names have changed to understandable column names


['GroupByThenStd(fruitset,andrena)',
 'GroupByThenStd(seeds,andrena)',
 'GroupByThenStd(fruitmass,bumbles)',
 'GroupByThenStd(fruitmass,AverageRainingDays)',
 'GroupByThenStd(seeds,RainingDays)',
 'GroupByThenStd(seeds,bumbles)',
 'GroupByThenStd(fruitset,AverageRainingDays)',
 'GroupByThenStd(fruitset,bumbles)',
 'GroupByThenStd(fruitset,RainingDays)',
 'GroupByThenStd(seeds,MaxOfLowerTRange)']

In [64]:
std_by_group = X_train.groupby('fruitset')['andrena'].std()
# std_by_group = X_train.groupby('fruitset')['andrena'].count()
# std_by_group = X_train.groupby('fruitset')['andrena'].mean()
std_by_group

fruitset
0.192732         NaN
0.226568         NaN
0.233554    0.110567
0.235916         NaN
0.237874         NaN
              ...   
0.642882    0.137155
0.644329    0.104003
0.645475    0.112229
0.645641    0.095185
0.652144    0.151052
Name: andrena, Length: 1526, dtype: float64

*All the columns with NaN are GroupByThenStd because those group contains less than two data points thus the standard deviation cannot be calculated so I think we can drop them*

In [65]:
# Drop all the columns with NaN
train_ofe_clean = train_ofe.drop(nan_cols, axis=1)
test_ofe_clean = test_ofe.drop(nan_cols, axis=1)
train_ofe.shape, test_ofe.shape, train_ofe_clean.shape, test_ofe_clean.shape

((15289, 320), (10194, 320), (15289, 310), (10194, 310))

In [90]:
# Convert numerical_column to numeric
for column in list(train_ofe_clean.columns):
    train_ofe_clean[column] = pd.to_numeric(train_ofe_clean[column])
    test_ofe_clean[column] = pd.to_numeric(test_ofe_clean[column])

In [95]:
models = [
    CatBoostRegressor(random_state=5, verbose=False, early_stopping_rounds=100),
    ExtraTreesRegressor(random_state=5),
    # GaussianNB(),
    HistGradientBoostingRegressor(random_state=5),
    knn_pipeline,
    linear_pipeline,
    LGBMRegressor(n_jobs=-1, random_state=5),
    LinearRegression(),
    RandomForestRegressor(random_state=5),
    XGBRegressor(random_state=5),
]

In [96]:
openfe_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    openfe_features[model_name] = list(train_ofe_clean.columns)

In [97]:
with open('openfe_features_ofe.txt', mode='w') as f:
    pprint(openfe_features, stream=f)

In [98]:
%%time

openfe_models = evaluate_models(models, train_ofe_clean, y_train, openfe_features, k10, f'{experiment_name}')
openfe_models

Models:   0%|          | 0/9 [00:00<?, ?it/s]

Progress:   0%|          | 0/9 [00:00<?, ?it/s]

Done with KNN.
Done with LR Pipeline.
Done with HistGradientBoostingRegressor.
Done with XGBRegressor.
Done with LGBMRegressor.
Done with LinearRegression.
Done with CatBoostRegressor.
Done with ExtraTreesRegressor.
Done with RandomForestRegressor.
CPU times: total: 8.3 s
Wall time: 5min 21s


,Model Name,Model Parameters,Model Train Accuracy,Model Test Accuracy,Model Test Accuracy Std,Model Time
2,HistGradientBoostingRegressor,"{'categorical_features': None, 'early_stopping...",306.723420,3.566724e+02,6.843644e-01,0 min 10.85 sec
5,LGBMRegressor,"{'boosting_type': 'gbdt', 'class_weight': None...",284.582041,3.578284e+02,1.218173e+00,0 min 8.60 sec
0,CatBoostRegressor,"{'loss_function': 'RMSE', 'verbose': False, 'r...",264.860066,3.603538e+02,5.975176e-01,2 min 3.34 sec
7,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",135.521288,3.638220e+02,5.022381e-01,3 min 38.52 sec
1,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.088081,3.756141e+02,1.906990e+00,3 min 23.77 sec
8,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",189.621524,3.825640e+02,2.250403e+00,0 min 45.15 sec
3,KNN,"{'memory': None, 'steps': [('scaler', Standard...",480.787673,4.918325e+02,1.305807e+00,0 min 0.38 sec
6,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",355.915066,3.604325e+08,4.302740e+08,0 min 1.35 sec
4,LR Pipeline,"{'memory': None, 'steps': [('scaler', Standard...",356.058696,5.028766e+12,1.150295e+12,0 min 0.93 sec


# FEATURE SELECTION

### Mutual Information

In [99]:
# To ensure the same randomness everytime
np.random.seed(5)

X_mi = train_ofe_clean.copy()

# Add random feature
X_mi['random_feature_continous'] = np.round(np.random.uniform(-2, 2, X_train.shape[0]), 6)
X_mi['random_feature_categorical'] = np.random.randint(1, 8, X_train.shape[0])
X_mi.head()

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,autoFE_f_0,autoFE_f_1,autoFE_f_2,autoFE_f_3,autoFE_f_4,autoFE_f_5,autoFE_f_6,autoFE_f_7,autoFE_f_8,autoFE_f_9,autoFE_f_10,autoFE_f_11,autoFE_f_12,autoFE_f_13,autoFE_f_14,autoFE_f_15,autoFE_f_16,autoFE_f_17,autoFE_f_18,autoFE_f_19,autoFE_f_20,autoFE_f_21,autoFE_f_22,autoFE_f_23,autoFE_f_24,autoFE_f_25,autoFE_f_26,autoFE_f_27,autoFE_f_28,autoFE_f_29,autoFE_f_30,autoFE_f_31,autoFE_f_32,autoFE_f_33,autoFE_f_34,autoFE_f_35,autoFE_f_36,autoFE_f_37,autoFE_f_38,autoFE_f_39,autoFE_f_40,autoFE_f_41,autoFE_f_42,autoFE_f_43,autoFE_f_44,autoFE_f_45,autoFE_f_46,autoFE_f_47,autoFE_f_48,autoFE_f_49,autoFE_f_50,autoFE_f_51,autoFE_f_52,autoFE_f_53,autoFE_f_54,autoFE_f_55,autoFE_f_56,autoFE_f_57,autoFE_f_58,autoFE_f_59,autoFE_f_60,autoFE_f_61,autoFE_f_62,autoFE_f_63,autoFE_f_64,autoFE_f_65,autoFE_f_66,autoFE_f_67,autoFE_f_68,autoFE_f_69,autoFE_f_70,autoFE_f_71,autoFE_f_72,autoFE_f_73,autoFE_f_74,autoFE_f_75,autoFE_f_76,autoFE_f_77,autoFE_f_78,autoFE_f_79,autoFE_f_80,autoFE_f_81,autoFE_f_82,autoFE_f_83,autoFE_f_84,autoFE_f_85,autoFE_f_86,autoFE_f_87,autoFE_f_88,autoFE_f_89,autoFE_f_90,autoFE_f_91,autoFE_f_92,autoFE_f_94,autoFE_f_95,autoFE_f_96,autoFE_f_97,autoFE_f_98,autoFE_f_99,autoFE_f_100,autoFE_f_102,autoFE_f_103,autoFE_f_104,autoFE_f_105,autoFE_f_106,autoFE_f_107,autoFE_f_108,autoFE_f_109,autoFE_f_110,autoFE_f_111,autoFE_f_112,autoFE_f_113,autoFE_f_114,autoFE_f_115,autoFE_f_116,autoFE_f_118,autoFE_f_119,autoFE_f_121,autoFE_f_122,autoFE_f_123,autoFE_f_124,autoFE_f_125,autoFE_f_127,autoFE_f_128,autoFE_f_129,autoFE_f_130,autoFE_f_131,autoFE_f_132,autoFE_f_133,autoFE_f_134,autoFE_f_135,autoFE_f_136,autoFE_f_137,autoFE_f_138,autoFE_f_139,autoFE_f_140,autoFE_f_141,autoFE_f_142,autoFE_f_143,autoFE_f_144,autoFE_f_145,autoFE_f_147,autoFE_f_148,autoFE_f_149,autoFE_f_150,autoFE_f_151,autoFE_f_153,autoFE_f_154,autoFE_f_155,autoFE_f_156,autoFE_f_157,autoFE_f_158,autoFE_f_159,autoFE_f_160,autoFE_f_161,autoFE_f_162,autoFE_f_163,autoFE_f_164,autoFE_f_166,autoFE_f_167,autoFE_f_168,autoFE_f_169,autoFE_f_170,autoFE_f_171,autoFE_f_172,autoFE_f_173,autoFE_f_174,autoFE_f_175,autoFE_f_176,autoFE_f_177,autoFE_f_178,autoFE_f_179,autoFE_f_180,autoFE_f_181,autoFE_f_182,autoFE_f_183,autoFE_f_184,autoFE_f_185,autoFE_f_186,autoFE_f_187,autoFE_f_188,autoFE_f_189,autoFE_f_190,autoFE_f_191,autoFE_f_192,autoFE_f_193,autoFE_f_194,autoFE_f_195,autoFE_f_196,autoFE_f_197,autoFE_f_198,autoFE_f_199,autoFE_f_200,autoFE_f_201,autoFE_f_202,autoFE_f_203,autoFE_f_204,autoFE_f_205,autoFE_f_206,autoFE_f_207,autoFE_f_208,autoFE_f_209,autoFE_f_210,autoFE_f_211,autoFE_f_212,autoFE_f_213,autoFE_f_214,autoFE_f_215,autoFE_f_216,autoFE_f_217,autoFE_f_218,autoFE_f_219,autoFE_f_220,autoFE_f_221,autoFE_f_224,autoFE_f_225,autoFE_f_226,autoFE_f_227,autoFE_f_228,autoFE_f_229,autoFE_f_230,autoFE_f_231,autoFE_f_232,autoFE_f_233,autoFE_f_234,autoFE_f_235,autoFE_f_236,autoFE_f_237,autoFE_f_238,autoFE_f_239,autoFE_f_240,autoFE_f_241,autoFE_f_242,autoFE_f_243,autoFE_f_244,autoFE_f_245,autoFE_f_246,autoFE_f_247,autoFE_f_248,autoFE_f_249,autoFE_f_250,autoFE_f_251,autoFE_f_252,autoFE_f_253,autoFE_f_254,autoFE_f_255,autoFE_f_256,autoFE_f_257,autoFE_f_258,autoFE_f_259,autoFE_f_260,autoFE_f_261,autoFE_f_262,autoFE_f_263,autoFE_f_264,autoFE_f_265,autoFE_f_266,autoFE_f_267,autoFE_f_268,autoFE_f_269,autoFE_f_270,autoFE_f_271,autoFE_f_272,autoFE_f_273,autoFE_f_274,autoFE_f_275,autoFE_f_276,autoFE_f_277,autoFE_f_278,autoFE_f_279,autoFE_f_280,autoFE_f_281,autoFE_f_282,autoFE_f_283,autoFE_f_284,autoFE_f_285,autoFE_f_286,autoFE_f_287,autoFE_f_288,autoFE_f_289,autoFE_f_290,autoFE_f_291,autoFE_f_292,autoFE_f_293,autoFE_f_294,autoFE_f_295,autoFE_f_296,autoFE_f_297,autoFE_f_298,autoFE_f_299,autoFE_f_300,autoFE_f_301,autoFE_f_302,autoFE_f_303,random_feature_continous,random_feature_categorical
0,25.0,0.50,0.25,0.75,0.50,

In [100]:
# Combine X and y for mutual information

X_and_y = pd.concat([X_mi, y_train], axis=1)

In [101]:
# Initialize parameters
random_states = [5, 42, 100, 500]
n_neighbors_list = [3, 5, 7, 10, 20]
results = defaultdict(list)

In [102]:
# Calculate MI for each combination of random_state and n_neighbors
for random_state in random_states:
    for n_neighbors in n_neighbors_list:        
        # Calculate MI
        mi = mutual_info_regression(X_and_y, y_train, n_neighbors=n_neighbors, random_state=random_state)
        
        # Store results if the target has the highest MI score
        mi_dict = dict(zip(X_and_y.columns, mi))
        if mi_dict[TARGET] == max(mi_dict.values()):
            for feature, score in mi_dict.items():
                results[feature].append(score)

        print(f'Done with Random State - {random_state} and N Neighbors - {n_neighbors}')

# 16 minutes

Done with Random State - 5 and N Neighbors - 3
Done with Random State - 5 and N Neighbors - 5
Done with Random State - 5 and N Neighbors - 7
Done with Random State - 5 and N Neighbors - 10
Done with Random State - 5 and N Neighbors - 20
Done with Random State - 42 and N Neighbors - 3
Done with Random State - 42 and N Neighbors - 5
Done with Random State - 42 and N Neighbors - 7
Done with Random State - 42 and N Neighbors - 10
Done with Random State - 42 and N Neighbors - 20
Done with Random State - 100 and N Neighbors - 3
Done with Random State - 100 and N Neighbors - 5
Done with Random State - 100 and N Neighbors - 7
Done with Random State - 100 and N Neighbors - 10
Done with Random State - 100 and N Neighbors - 20
Done with Random State - 500 and N Neighbors - 3
Done with Random State - 500 and N Neighbors - 5
Done with Random State - 500 and N Neighbors - 7
Done with Random State - 500 and N Neighbors - 10
Done with Random State - 500 and N Neighbors - 20


In [103]:
# Average MI scores across valid combinations
average_mi = {feature: np.mean(scores) for feature, scores in results.items() if scores}
average_mi

{'clonesize': 0.09688619996819144,
 'honeybee': 0.06836308197807399,
 'bumbles': 0.02262557407668959,
 'andrena': 0.026954120386921753,
 'osmia': 0.03508373077921008,
 'MaxOfUpperTRange': 0.01804195616734954,
 'MinOfUpperTRange': 0.016968723538207574,
 'AverageOfUpperTRange': 0.017749899770761867,
 'MaxOfLowerTRange': 0.01799892497265825,
 'MinOfLowerTRange': 0.015239468362257601,
 'AverageOfLowerTRange': 0.01726033077238196,
 'RainingDays': 0.16272430034888377,
 'AverageRainingDays': 0.16330437873593276,
 'fruitset': 1.250255131825028,
 'fruitmass': 0.9069587932775496,
 'seeds': 1.0754376689271432,
 'autoFE_f_0': 1.1619198863622875,
 'autoFE_f_1': 1.2503169122808784,
 'autoFE_f_2': 1.0882098213032207,
 'autoFE_f_3': 0.900203945723483,
 'autoFE_f_4': 1.0302142696544496,
 'autoFE_f_5': 1.080281991501603,
 'autoFE_f_6': 1.0754587191842613,
 'autoFE_f_7': 0.9595427641320979,
 'autoFE_f_8': 0.9592901093945503,
 'autoFE_f_9': 0.9595640190377228,
 'autoFE_f_10': 1.1017991184942004,
 'autoFE_

In [104]:
# Display results
sorted_mi = sorted(average_mi.items(), key=lambda x: x[1], reverse=True)
print("Average MI scores:")
for item in sorted_mi:
    print(item)

Average MI scores:
('yield', 6.590818904098439)
('autoFE_f_1', 1.2503169122808784)
('fruitset', 1.250255131825028)
('autoFE_f_30', 1.1927776667202483)
('autoFE_f_0', 1.1619198863622875)
('autoFE_f_49', 1.1061045145802701)
('autoFE_f_10', 1.1017991184942004)
('autoFE_f_2', 1.0882098213032207)
('autoFE_f_5', 1.080281991501603)
('autoFE_f_29', 1.0786695749067392)
('autoFE_f_25', 1.07582081746041)
('autoFE_f_6', 1.0754587191842613)
('seeds', 1.0754376689271432)
('autoFE_f_4', 1.0302142696544496)
('autoFE_f_15', 1.0285572346958207)
('autoFE_f_43', 0.9958241259952103)
('autoFE_f_35', 0.9792922804176782)
('autoFE_f_19', 0.9742922133803926)
('autoFE_f_9', 0.9595640190377228)
('autoFE_f_7', 0.9595427641320979)
('autoFE_f_44', 0.9594685241199589)
('autoFE_f_8', 0.9592901093945503)
('autoFE_f_97', 0.9590318049745765)
('autoFE_f_21', 0.9206289920978028)
('fruitmass', 0.9069587932775496)
('autoFE_f_54', 0.9066776805792711)
('autoFE_f_3', 0.900203945723483)
('autoFE_f_12', 0.8998850956256014)
('auto

In [105]:
# Determine higher MI between 0 and random_feature
higher_threshold = max(0, average_mi.get('random_feature_categorical', 0), average_mi.get('random_feature_continous', 0))
higher_threshold

0.0024292609350075045

In [106]:
# List features with MI higher than the threshold, excluding the target
mi_features_list = [feature for feature, score in sorted_mi if feature != TARGET and score > higher_threshold]
mi_features_list

['autoFE_f_1',
 'fruitset',
 'autoFE_f_30',
 'autoFE_f_0',
 'autoFE_f_49',
 'autoFE_f_10',
 'autoFE_f_2',
 'autoFE_f_5',
 'autoFE_f_29',
 'autoFE_f_25',
 'autoFE_f_6',
 'seeds',
 'autoFE_f_4',
 'autoFE_f_15',
 'autoFE_f_43',
 'autoFE_f_35',
 'autoFE_f_19',
 'autoFE_f_9',
 'autoFE_f_7',
 'autoFE_f_44',
 'autoFE_f_8',
 'autoFE_f_97',
 'autoFE_f_21',
 'fruitmass',
 'autoFE_f_54',
 'autoFE_f_3',
 'autoFE_f_12',
 'autoFE_f_290',
 'autoFE_f_13',
 'autoFE_f_67',
 'autoFE_f_16',
 'autoFE_f_11',
 'autoFE_f_40',
 'autoFE_f_22',
 'autoFE_f_20',
 'autoFE_f_91',
 'autoFE_f_59',
 'autoFE_f_107',
 'autoFE_f_50',
 'autoFE_f_39',
 'autoFE_f_47',
 'autoFE_f_36',
 'autoFE_f_37',
 'autoFE_f_42',
 'autoFE_f_46',
 'autoFE_f_32',
 'autoFE_f_41',
 'autoFE_f_48',
 'autoFE_f_14',
 'autoFE_f_23',
 'autoFE_f_26',
 'autoFE_f_90',
 'autoFE_f_27',
 'autoFE_f_34',
 'autoFE_f_24',
 'autoFE_f_17',
 'autoFE_f_38',
 'autoFE_f_294',
 'autoFE_f_277',
 'autoFE_f_53',
 'autoFE_f_18',
 'autoFE_f_257',
 'autoFE_f_71',
 'autoFE

In [107]:
mi_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    mi_features[model_name] = mi_features_list

In [108]:
with open('mi_features_ofe.txt', mode='w') as f:
    pprint(mi_features, stream=f)

In [109]:
%%time

openfe_models = evaluate_models(models, train_ofe_clean, y_train, mi_features, k10, f'{experiment_name}')
openfe_models

Models:   0%|          | 0/9 [00:00<?, ?it/s]

Progress:   0%|          | 0/9 [00:00<?, ?it/s]

Done with HistGradientBoostingRegressor.
Done with LGBMRegressor.
Done with LinearRegression.
Done with LR Pipeline.
Done with KNN.
Done with CatBoostRegressor.
Done with XGBRegressor.
Done with ExtraTreesRegressor.
Done with RandomForestRegressor.
CPU times: total: 6.44 s
Wall time: 4min 42s


,Model Name,Model Parameters,Model Train Accuracy,Model Test Accuracy,Model Test Accuracy Std,Model Time
2,HistGradientBoostingRegressor,"{'categorical_features': None, 'early_stopping...",312.520238,3.558268e+02,1.391191e+00,0 min 9.11 sec
5,LGBMRegressor,"{'boosting_type': 'gbdt', 'class_weight': None...",284.871166,3.573111e+02,1.465843e+00,0 min 6.94 sec
0,CatBoostRegressor,"{'loss_function': 'RMSE', 'verbose': False, 'r...",266.005446,3.587521e+02,1.414994e+00,1 min 40.13 sec
7,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",135.532380,3.642212e+02,6.090605e-01,3 min 22.67 sec
1,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.088081,3.756752e+02,1.751338e+00,3 min 5.06 sec
8,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",189.621524,3.812772e+02,2.003249e+00,0 min 41.27 sec
3,KNN,"{'memory': None, 'steps': [('scaler', Standard...",480.239929,4.910762e+02,1.315750e+00,0 min 0.39 sec
6,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",355.916883,4.370177e+03,2.302077e+03,0 min 0.72 sec
4,LR Pipeline,"{'memory': None, 'steps': [('scaler', Standard...",356.233067,2.273845e+12,1.450825e+12,0 min 0.77 sec


### Permutation Importance

In [133]:
%%time

perm_cv = KFold(n_splits=3, shuffle=True, random_state=5)

perm_importances = {model.name if hasattr(model, 'name') else model.__class__.__name__: [] for model in models}

for i, (train_idx, test_idx) in enumerate(perm_cv.split(X_mi, y_train)):
    X_train_pi, X_test_pi = X_mi.iloc[train_idx], X_mi.iloc[test_idx]
    y_train_pi, y_test_pi = y_train.iloc[train_idx], y_train.iloc[test_idx]

    for model in models:
        if hasattr(model, 'name'):
            model_name = model.name
        else:
            model_name = model.__class__.__name__

        model.fit(X_train_pi, y_train_pi)

        # Calculate permutation importance
        result = permutation_importance(model, X_test_pi, y_test_pi, n_repeats=5, random_state=5, scoring='neg_mean_absolute_error')

        perm_importances[model_name].append(result.importances_mean)
        
        print(f'Done with {model_name}.')
    
    print(f'Done with Fold {i+1}', end='\n\n')

Done with CatBoostRegressor.
Done with ExtraTreesRegressor.
Done with HistGradientBoostingRegressor.
Done with KNN.
Done with LR Pipeline.
Done with LGBMRegressor.
Done with LinearRegression.
Done with RandomForestRegressor.
Done with XGBRegressor.
Done with Fold 1

Done with CatBoostRegressor.
Done with ExtraTreesRegressor.
Done with HistGradientBoostingRegressor.
Done with KNN.
Done with LR Pipeline.
Done with LGBMRegressor.
Done with LinearRegression.
Done with RandomForestRegressor.
Done with XGBRegressor.
Done with Fold 2

Done with CatBoostRegressor.
Done with ExtraTreesRegressor.
Done with HistGradientBoostingRegressor.
Done with KNN.
Done with LR Pipeline.
Done with LGBMRegressor.
Done with LinearRegression.
Done with RandomForestRegressor.
Done with XGBRegressor.
Done with Fold 3

CPU times: total: 3h 51min 3s
Wall time: 3h 19min 21s


In [134]:
%%time

# Average importances across folds and export to CSV
for model_name, importances in perm_importances.items():
    avg_importance = np.mean(importances, axis=0)

    importance_df = pd.DataFrame({'Feature': X_mi.columns, 'Importance': avg_importance})

    importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    
    # Export to CSV
    importance_df.to_csv(f'.\permutation_importances\{model_name}_permutation_importance.csv', index=False)

print('Done with Permuation Importances', end='\n\n')

Done with Permuation Importances

CPU times: total: 15.6 ms
Wall time: 51.7 ms


In [135]:
directory = 'permutation_importances'

# Initialize a dictionary for the features
perm_important_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__
    print(f'Model: {model_name}')

    csv_path = os.path.join(directory, f'{model_name}_permutation_importance.csv')
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)

        # Initialize importance variables
        random_feature_importance_cont = 0
        random_feature_importance_cat = 0
        
        # Check for 'random_feature_continous' and its importance
        if 'random_feature_continous' in df['Feature'].values:
            random_feature_importance_cont = df.loc[df['Feature'] == 'random_feature_continous', 'Importance'].iloc[0]
            print(random_feature_importance_cont)
        if 'random_feature_categorical' in df['Feature'].values:
            random_feature_importance_cat = df.loc[df['Feature'] == 'random_feature_categorical', 'Importance'].iloc[0]
            print(random_feature_importance_cat)
        else:
            random_feature_importance = 0

        # Determine the threshold
        threshold = max(0, random_feature_importance_cont, random_feature_importance_cat)
        print(f'Threshold: {threshold}')

        # Filter features where importance is greater than 0
        important_feats_filtered = df[df['Importance'] > threshold]['Feature'].tolist()

        # # Reorder important_feats based on the predefined features_list
        # important_feats_ordered = [feat for feat in features_list if feat in important_feats_filtered]

        # Add to importance dictionary
        perm_important_features[model_name] = important_feats_filtered

    else:
        print(f'CSV file for {model_name} not found.')

print('Done getting important features dictionary')

Model: CatBoostRegressor
0.6305742846787629
-0.1101050176316259
Threshold: 0.6305742846787629
Model: ExtraTreesRegressor
-0.1687513604265859
0.2108935119996772
Threshold: 0.2108935119996772
Model: HistGradientBoostingRegressor
-0.0386719684141401
-0.0067297981883181
Threshold: 0
Model: KNN
0.1100206248867418
0.26540902566461
Threshold: 0.26540902566461
Model: LR Pipeline
-0.0399902343749999
-0.0050455729166666
Threshold: 0
Model: LGBMRegressor
-0.2628713371120587
0.2500800615004095
Threshold: 0.2500800615004095
Model: LinearRegression
-0.03503130103151
-0.0046430629988511
Threshold: 0
Model: RandomForestRegressor
-0.2115225162797931
0.0184743937541914
Threshold: 0.0184743937541914
Model: XGBRegressor
-0.8572510388749529
0.1742902170630752
Threshold: 0.1742902170630752
Done getting important features dictionary


In [136]:
with open('perm_important_features_ofe.txt', mode='w') as f:
    pprint(perm_important_features, stream=f)

In [137]:
%%time

perm_imp_models = evaluate_models(models, train_ofe_clean, y_train, perm_important_features, k10, f'{experiment_name}')
perm_imp_models

Models:   0%|          | 0/9 [00:00<?, ?it/s]

Progress:   0%|          | 0/9 [00:00<?, ?it/s]

Done with LGBMRegressor.
Done with KNN.
Done with LinearRegression.
Done with HistGradientBoostingRegressor.
Done with ExtraTreesRegressor.
Done with XGBRegressor.
Done with LR Pipeline.
Done with CatBoostRegressor.
Done with RandomForestRegressor.
CPU times: total: 2.91 s
Wall time: 2min 1s


,Model Name,Model Parameters,Model Train Accuracy,Model Test Accuracy,Model Test Accuracy Std,Model Time
2,HistGradientBoostingRegressor,"{'categorical_features': None, 'early_stopping...",308.360755,3.558238e+02,1.007208e+00,0 min 4.08 sec
5,LGBMRegressor,"{'boosting_type': 'gbdt', 'class_weight': None...",289.619349,3.567973e+02,1.512244e+00,0 min 1.95 sec
0,CatBoostRegressor,"{'loss_function': 'RMSE', 'verbose': False, 'r...",271.048756,3.584085e+02,1.674231e+00,1 min 0.92 sec
3,KNN,"{'memory': None, 'steps': [('scaler', Standard...",355.597089,3.623269e+02,3.230285e+00,0 min 0.08 sec
7,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",135.340409,3.641293e+02,6.057182e-01,1 min 20.91 sec
1,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.088081,3.715439e+02,1.258526e+00,0 min 35.83 sec
8,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",190.061984,3.811891e+02,8.297489e-01,0 min 23.65 sec
6,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",355.911991,2.288999e+08,2.320232e+08,0 min 0.81 sec
4,LR Pipeline,"{'memory': None, 'steps': [('scaler', Standard...",356.490587,1.203362e+13,8.213508e+12,0 min 0.80 sec


### SFS

In [ ]:
%%time

# Initialize empty dictionary for SFS features
sfs_features = {}

for model in models:
    if hasattr(model, 'name'):
        model_name = model.name
    else:
        model_name = model.__class__.__name__

    try:
        # features = feats_so_far[model_name]
        features = baseline_features[model_name]

        # incase there is no feature that had importance, go to the next model
        if len(features) == 0:
            continue
        
        X_sfs = X_train[features]

        print(f'Running backward feature selection with {model_name}')

        sfs = SFS(model,
            k_features='best',
            forward=False,
            floating=False,
            scoring='neg_mean_absolute_error',
            verbose=2,
            n_jobs=-1,
            cv=k10)
        
        sfs = sfs.fit(X_sfs, y_train)

        # Get the selected features index
        selected_sfs_idx = list(sfs.k_feature_idx_)

        # Get the feature names
        selected_sfs_feats = X_sfs.columns[selected_sfs_idx]

        selected_features = list(selected_sfs_feats)

        # # Reorder selected_features based on the predefined features_list
        # selected_features_ordered = [feat for feat in features_list if feat in selected_features]

        sfs_features[model_name] = selected_features

        print(f'Done with {model_name}', end='\n\n')

    except KeyError:
        print(f'{model_name} not in the dictionary.')

# Took 138 minutes

In [ ]:
%%time

sfs_models = evaluate_models(models, X_train, y_train, sfs_features, k10, f'{experiment_name}_sfs')
sfs_models

In [ ]:
with open('sfs_features.txt', mode='w') as f:
    pprint(sfs_features, stream=f)

In [ ]:
sfs_features = {'CatBoostRegressor': ['honeybee',
                       'MaxOfUpperTRange',
                       'MinOfUpperTRange',
                       'AverageOfUpperTRange',
                       'MinOfLowerTRange',
                       'AverageOfLowerTRange',
                       'AverageRainingDays',
                       'fruitset',
                       'fruitmass',
                       'seeds'],
 'ExtraTreesRegressor': ['clonesize',
                         'andrena',
                         'osmia',
                         'AverageOfLowerTRange',
                         'RainingDays',
                         'fruitset',
                         'fruitmass',
                         'seeds'],
 'HistGradientBoostingRegressor': ['clonesize',
                                   'honeybee',
                                   'MinOfUpperTRange',
                                   'RainingDays',
                                   'fruitset',
                                   'fruitmass',
                                   'seeds'],
 'KNN': ['MaxOfLowerTRange', 'fruitset', 'seeds'],
 'LGBMRegressor': ['clonesize',
                   'MaxOfLowerTRange',
                   'RainingDays',
                   'AverageRainingDays',
                   'fruitset',
                   'fruitmass',
                   'seeds'],
 'LR Pipeline': ['bumbles',
                 'andrena',
                 'osmia',
                 'AverageOfUpperTRange',
                 'MinOfLowerTRange',
                 'AverageRainingDays',
                 'fruitset',
                 'fruitmass',
                 'seeds'],
 'LinearRegression': ['bumbles',
                      'andrena',
                      'osmia',
                      'AverageOfUpperTRange',
                      'MinOfLowerTRange',
                      'AverageRainingDays',
                      'fruitset',
                      'fruitmass',
                      'seeds'],
 'RandomForestRegressor': ['clonesize',
                           'bumbles',
                           'andrena',
                           'osmia',
                           'MaxOfUpperTRange',
                           'MinOfUpperTRange',
                           'AverageOfUpperTRange',
                           'MinOfLowerTRange',
                           'AverageOfLowerTRange',
                           'RainingDays',
                           'AverageRainingDays',
                           'fruitset',
                           'fruitmass',
                           'seeds'],
 'XGBRegressor': ['AverageOfLowerTRange',
                  'AverageRainingDays',
                  'fruitset',
                  'seeds']}


## Best Single Model

In [110]:
model = HistGradientBoostingRegressor(random_state=5)

model.fit(train_ofe_clean[mi_features_list], y_train)

HistGradientBoostingRegressor(random_state=5)

In [111]:
pred = model.predict(test_ofe_clean[mi_features_list])

pred_df = pd.DataFrame(pred, columns=['yield'])
pred_df.head()

,yield
0,4251.973443
1,6036.019978
2,7239.748855
3,4729.879767
4,4211.327659


In [121]:
test_pred = pd.concat([test_ofe_clean[mi_features_list], pred_df], axis=1)
test_pred.head()

,autoFE_f_1,fruitset,autoFE_f_30,autoFE_f_0,autoFE_f_49,autoFE_f_10,autoFE_f_2,autoFE_f_5,autoFE_f_29,autoFE_f_25,autoFE_f_6,seeds,autoFE_f_4,autoFE_f_15,autoFE_f_43,autoFE_f_35,autoFE_f_19,autoFE_f_9,autoFE_f_7,autoFE_f_44,autoFE_f_8,autoFE_f_97,autoFE_f_21,fruitmass,autoFE_f_54,autoFE_f_3,autoFE_f_12,autoFE_f_290,autoFE_f_13,autoFE_f_67,autoFE_f_16,autoFE_f_11,autoFE_f_40,autoFE_f_22,autoFE_f_20,autoFE_f_91,autoFE_f_59,autoFE_f_107,autoFE_f_50,autoFE_f_39,autoFE_f_47,autoFE_f_36,autoFE_f_37,autoFE_f_42,autoFE_f_46,autoFE_f_32,autoFE_f_41,autoFE_f_48,autoFE_f_14,autoFE_f_23,autoFE_f_26,autoFE_f_90,autoFE_f_27,autoFE_f_34,autoFE_f_24,autoFE_f_17,autoFE_f_38,autoFE_f_294,autoFE_f_277,autoFE_f_53,autoFE_f_18,autoFE_f_257,autoFE_f_71,autoFE_f_270,autoFE_f_63,autoFE_f_28,autoFE_f_62,autoFE_f_157,autoFE_f_57,autoFE_f_123,autoFE_f_31,autoFE_f_98,autoFE_f_68,autoFE_f_269,autoFE_f_184,autoFE_f_227,autoFE_f_192,autoFE_f_66,autoFE_f_235,autoFE_f_119,autoFE_f_229,autoFE_f_245,autoFE_f_84,autoFE_f_143,autoFE_f_260,autoFE_f_51,autoFE_f_291,autoFE_f_262,autoFE_f_212,autoFE_f_142,autoFE_f_83,autoFE_f_228,autoFE_f_100,autoFE_f_138,autoFE_f_111,autoFE_f_122,autoFE_f_205,autoFE_f_299,autoFE_f_239,autoFE_f_190,autoFE_f_296,AverageRainingDays,RainingDays,autoFE_f_125,autoFE_f_219,autoFE_f_147,autoFE_f_61,autoFE_f_88,autoFE_f_238,autoFE_f_183,autoFE_f_287,autoFE_f_77,autoFE_f_60,autoFE_f_129,autoFE_f_211,autoFE_f_244,autoFE_f_216,autoFE_f_65,autoFE_f_274,autoFE_f_226,autoFE_f_255,autoFE_f_248,autoFE_f_55,autoFE_f_109,autoFE_f_108,autoFE_f_86,autoFE_f_70,autoFE_f_103,autoFE_f_72,autoFE_f_276,autoFE_f_166,autoFE_f_169,autoFE_f_263,autoFE_f_214,autoFE_f_224,autoFE_f_187,clonesize,autoFE_f_95,autoFE_f_196,autoFE_f_80,autoFE_f_106,autoFE_f_256,autoFE_f_168,autoFE_f_182,autoFE_f_75,autoFE_f_249,autoFE_f_45,autoFE_f_207,autoFE_f_280,autoFE_f_159,autoFE_f_151,autoFE_f_272,autoFE_f_137,autoFE_f_78,autoFE_f_279,autoFE_f_251,autoFE_f_177,honeybee,autoFE_f_131,autoFE_f_288,autoFE_f_267,autoFE_f_203,autoFE_f_233,autoFE_f_201,autoFE_f_114,autoFE_f_56,autoFE_f_81,autoFE_f_188,autoFE_f_99,autoFE_f_230,autoFE_f_225,autoFE_f_94,autoFE_f_234,autoFE_f_105,autoFE_f_85,autoFE_f_292,autoFE_f_52,autoFE_f_174,autoFE_f_221,autoFE_f_197,autoFE_f_69,autoFE_f_161,autoFE_f_281,osmia,autoFE_f_92,autoFE_f_73,autoFE_f_162,autoFE_f_181,autoFE_f_191,autoFE_f_302,autoFE_f_210,autoFE_f_289,autoFE_f_295,andrena,autoFE_f_217,autoFE_f_96,autoFE_f_215,autoFE_f_259,autoFE_f_213,bumbles,autoFE_f_113,autoFE_f_64,autoFE_f_206,autoFE_f_271,autoFE_f_172,autoFE_f_264,autoFE_f_175,autoFE_f_303,autoFE_f_278,autoFE_f_127,autoFE_f_252,autoFE_f_133,autoFE_f_283,autoFE_f_116,autoFE_f_110,autoFE_f_156,autoFE_f_282,MaxOfUpperTRange,autoFE_f_253,MaxOfLowerTRange,autoFE_f_160,AverageOfUpperTRange,autoFE_f_163,autoFE_f_240,autoFE_f_237,autoFE_f_136,autoFE_f_193,autoFE_f_121,autoFE_f_173,autoFE_f_202,autoFE_f_115,autoFE_f_128,autoFE_f_254,autoFE_f_297,autoFE_f_180,autoFE_f_241,autoFE_f_261,AverageOfLowerTRange,autoFE_f_141,autoFE_f_243,autoFE_f_154,autoFE_f_300,autoFE_f_258,autoFE_f_74,autoFE_f_158,MinOfUpperTRange,autoFE_f_164,autoFE_f_145,autoFE_f_167,autoFE_f_194,autoFE_f_231,autoFE_f_170,MinOfLowerTRange,autoFE_f_275,autoFE_f_218,autoFE_f_298,autoFE_f_266,autoFE_f_33,autoFE_f_220,autoFE_f_153,autoFE_f_268,autoFE_f_236,autoFE_f_171,autoFE_f_232,autoFE_f_139,autoFE_f_79,autoFE_f_132,autoFE_f_209,autoFE_f_89,autoFE_f_208,autoFE_f_204,autoFE_f_178,autoFE_f_199,autoFE_f_242,autoFE_f_250,autoFE_f_247,autoFE_f_149,autoFE_f_134,autoFE_f_273,autoFE_f_148,autoFE_f_102,autoFE_f_112,autoFE_f_185,autoFE_f_198,autoFE_f_285,autoFE_f_104,autoFE_f_58,autoFE_f_186,autoFE_f_293,autoFE_f_150,autoFE_f_195,autoFE_f_82,autoFE_f_87,autoFE_f_200,autoFE_f_155,autoFE_f_124,autoFE_f_189,autoFE_f_246,autoFE_f_284,autoFE_f_179,autoFE_f_140,yield
0,0.399367,0.399367,0.408088,12.537955,0.807455,0.162977,31.793936,31.802657,-30.995202,-30.986480,31.394569,31.394569,31.0,0.123441,0.399367,0.251378,12.811749,0.109506,0.109490,0.109506,0

### Manual Correction

In [122]:
# Combine X and y for mutual information

X_and_y = pd.concat([train_ofe_clean[mi_features_list], y_train], axis=1)

In [123]:
# Get list of all feature columns (excluding TARGET)
feature_columns = X_and_y.columns[X_and_y.columns != TARGET]

# Initialize a dictionary to store corrections
corrections = {}

# Iterate over each feature column
for column in feature_columns:
    # Count occurrences of each value in the column
    value_counts = X_and_y[column].value_counts()

    # Filter values that appear at least twice
    values_to_check = value_counts[value_counts >= 2].index
     
    # Iterate over each value in the column that appears at least twice
    for value in values_to_check:
        # Get rows where column equals value
        rows_with_value = X_and_y[X_and_y[column] == value]

        # Check if all rows with this value in column have the same target
        if rows_with_value[TARGET].nunique() == 1:
            target_value = rows_with_value[TARGET].iloc[0]
        
            # Store correction if it matches training pattern
            if (column, value, target_value) not in corrections:
                corrections[(column, value)] = target_value

In [124]:
corrections

{('autoFE_f_1', 0.249334678): 2605.69676,
 ('autoFE_f_1', 0.552101998): 7198.42285,
 ('fruitset', 0.249334678): 2605.69676,
 ('fruitset', 0.552101998): 7198.42285,
 ('autoFE_f_30', 0.370487534): 3139.43255,
 ('autoFE_f_30', 0.552101998): 7198.42285,
 ('autoFE_f_30', 0.342841074): 3436.49354,
 ('autoFE_f_0', 26.010030712147667): 7825.87492,
 ('autoFE_f_0', 10.835292393834624): 3708.20502,
 ('autoFE_f_0', 5.837616892739071): 2605.69676,
 ('autoFE_f_0', 8.645255682729612): 3049.26032,
 ('autoFE_f_0', 26.97231373398639): 8086.38881,
 ('autoFE_f_0', 12.294650507036259): 4179.18592,
 ('autoFE_f_0', 9.453110791942418): 3501.17992,
 ('autoFE_f_0', 8.885980656230375): 3238.02815,
 ('autoFE_f_0', 15.259729951702418): 5046.45734,
 ('autoFE_f_0', 14.17534999427809): 4673.38052,
 ('autoFE_f_0', 14.860035087676671): 4947.60566,
 ('autoFE_f_0', 18.590380091715055): 6301.03442,
 ('autoFE_f_0', 14.339741262671172): 4503.16186,
 ('autoFE_f_0', 7.632562839677861): 2688.02883,
 ('autoFE_f_0', 19.824076153

In [125]:
# Initialize a dictionary to store feature names related to each target value
target_features_dict = {}

# Iterate over corrections to populate target_features_dict
for (column, value), target_value in corrections.items():
    if target_value not in target_features_dict:
        target_features_dict[target_value] = []
    target_features_dict[target_value].append(column)

# Print or return the dictionary where target values are keys and related feature names are values
print("Dictionary of target values and related feature names:")
print(target_features_dict)

Dictionary of target values and related feature names:
{2605.69676: ['autoFE_f_1', 'fruitset', 'autoFE_f_0', 'autoFE_f_49', 'autoFE_f_49', 'autoFE_f_10', 'autoFE_f_10', 'autoFE_f_2', 'autoFE_f_5', 'autoFE_f_5', 'autoFE_f_29', 'autoFE_f_25', 'autoFE_f_25', 'autoFE_f_15', 'autoFE_f_43', 'autoFE_f_35', 'autoFE_f_19', 'autoFE_f_19', 'autoFE_f_9', 'autoFE_f_9', 'autoFE_f_7', 'autoFE_f_7', 'autoFE_f_44', 'autoFE_f_44', 'autoFE_f_8', 'autoFE_f_8', 'autoFE_f_97', 'autoFE_f_97', 'autoFE_f_16', 'autoFE_f_16', 'autoFE_f_11', 'autoFE_f_11', 'autoFE_f_40', 'autoFE_f_22', 'autoFE_f_22', 'autoFE_f_39', 'autoFE_f_39', 'autoFE_f_39', 'autoFE_f_42', 'autoFE_f_42', 'autoFE_f_46', 'autoFE_f_32', 'autoFE_f_41', 'autoFE_f_48', 'autoFE_f_14', 'autoFE_f_14', 'autoFE_f_23'], 7198.42285: ['autoFE_f_1', 'fruitset', 'autoFE_f_30'], 3139.43255: ['autoFE_f_30', 'autoFE_f_49', 'autoFE_f_10', 'autoFE_f_5', 'autoFE_f_25', 'autoFE_f_19', 'autoFE_f_11', 'autoFE_f_22', 'autoFE_f_20', 'autoFE_f_91', 'autoFE_f_59', 'autoFE

In [127]:
# Apply corrections to test_pred
for (column, value), target_value in corrections.items():
    test_pred.loc[test_pred[column] == value, TARGET] = target_value

# Print corrected predictions
print("Corrected predictions in test_pred:")
test_pred

Corrected predictions in test_pred:


,autoFE_f_1,fruitset,autoFE_f_30,autoFE_f_0,autoFE_f_49,autoFE_f_10,autoFE_f_2,autoFE_f_5,autoFE_f_29,autoFE_f_25,autoFE_f_6,seeds,autoFE_f_4,autoFE_f_15,autoFE_f_43,autoFE_f_35,autoFE_f_19,autoFE_f_9,autoFE_f_7,autoFE_f_44,autoFE_f_8,autoFE_f_97,autoFE_f_21,fruitmass,autoFE_f_54,autoFE_f_3,autoFE_f_12,autoFE_f_290,autoFE_f_13,autoFE_f_67,autoFE_f_16,autoFE_f_11,autoFE_f_40,autoFE_f_22,autoFE_f_20,autoFE_f_91,autoFE_f_59,autoFE_f_107,autoFE_f_50,autoFE_f_39,autoFE_f_47,autoFE_f_36,autoFE_f_37,autoFE_f_42,autoFE_f_46,autoFE_f_32,autoFE_f_41,autoFE_f_48,autoFE_f_14,autoFE_f_23,autoFE_f_26,autoFE_f_90,autoFE_f_27,autoFE_f_34,autoFE_f_24,autoFE_f_17,autoFE_f_38,autoFE_f_294,autoFE_f_277,autoFE_f_53,autoFE_f_18,autoFE_f_257,autoFE_f_71,autoFE_f_270,autoFE_f_63,autoFE_f_28,autoFE_f_62,autoFE_f_157,autoFE_f_57,autoFE_f_123,autoFE_f_31,autoFE_f_98,autoFE_f_68,autoFE_f_269,autoFE_f_184,autoFE_f_227,autoFE_f_192,autoFE_f_66,autoFE_f_235,autoFE_f_119,autoFE_f_229,autoFE_f_245,autoFE_f_84,autoFE_f_143,autoFE_f_260,autoFE_f_51,autoFE_f_291,autoFE_f_262,autoFE_f_212,autoFE_f_142,autoFE_f_83,autoFE_f_228,autoFE_f_100,autoFE_f_138,autoFE_f_111,autoFE_f_122,autoFE_f_205,autoFE_f_299,autoFE_f_239,autoFE_f_190,autoFE_f_296,AverageRainingDays,RainingDays,autoFE_f_125,autoFE_f_219,autoFE_f_147,autoFE_f_61,autoFE_f_88,autoFE_f_238,autoFE_f_183,autoFE_f_287,autoFE_f_77,autoFE_f_60,autoFE_f_129,autoFE_f_211,autoFE_f_244,autoFE_f_216,autoFE_f_65,autoFE_f_274,autoFE_f_226,autoFE_f_255,autoFE_f_248,autoFE_f_55,autoFE_f_109,autoFE_f_108,autoFE_f_86,autoFE_f_70,autoFE_f_103,autoFE_f_72,autoFE_f_276,autoFE_f_166,autoFE_f_169,autoFE_f_263,autoFE_f_214,autoFE_f_224,autoFE_f_187,clonesize,autoFE_f_95,autoFE_f_196,autoFE_f_80,autoFE_f_106,autoFE_f_256,autoFE_f_168,autoFE_f_182,autoFE_f_75,autoFE_f_249,autoFE_f_45,autoFE_f_207,autoFE_f_280,autoFE_f_159,autoFE_f_151,autoFE_f_272,autoFE_f_137,autoFE_f_78,autoFE_f_279,autoFE_f_251,autoFE_f_177,honeybee,autoFE_f_131,autoFE_f_288,autoFE_f_267,autoFE_f_203,autoFE_f_233,autoFE_f_201,autoFE_f_114,autoFE_f_56,autoFE_f_81,autoFE_f_188,autoFE_f_99,autoFE_f_230,autoFE_f_225,autoFE_f_94,autoFE_f_234,autoFE_f_105,autoFE_f_85,autoFE_f_292,autoFE_f_52,autoFE_f_174,autoFE_f_221,autoFE_f_197,autoFE_f_69,autoFE_f_161,autoFE_f_281,osmia,autoFE_f_92,autoFE_f_73,autoFE_f_162,autoFE_f_181,autoFE_f_191,autoFE_f_302,autoFE_f_210,autoFE_f_289,autoFE_f_295,andrena,autoFE_f_217,autoFE_f_96,autoFE_f_215,autoFE_f_259,autoFE_f_213,bumbles,autoFE_f_113,autoFE_f_64,autoFE_f_206,autoFE_f_271,autoFE_f_172,autoFE_f_264,autoFE_f_175,autoFE_f_303,autoFE_f_278,autoFE_f_127,autoFE_f_252,autoFE_f_133,autoFE_f_283,autoFE_f_116,autoFE_f_110,autoFE_f_156,autoFE_f_282,MaxOfUpperTRange,autoFE_f_253,MaxOfLowerTRange,autoFE_f_160,AverageOfUpperTRange,autoFE_f_163,autoFE_f_240,autoFE_f_237,autoFE_f_136,autoFE_f_193,autoFE_f_121,autoFE_f_173,autoFE_f_202,autoFE_f_115,autoFE_f_128,autoFE_f_254,autoFE_f_297,autoFE_f_180,autoFE_f_241,autoFE_f_261,AverageOfLowerTRange,autoFE_f_141,autoFE_f_243,autoFE_f_154,autoFE_f_300,autoFE_f_258,autoFE_f_74,autoFE_f_158,MinOfUpperTRange,autoFE_f_164,autoFE_f_145,autoFE_f_167,autoFE_f_194,autoFE_f_231,autoFE_f_170,MinOfLowerTRange,autoFE_f_275,autoFE_f_218,autoFE_f_298,autoFE_f_266,autoFE_f_33,autoFE_f_220,autoFE_f_153,autoFE_f_268,autoFE_f_236,autoFE_f_171,autoFE_f_232,autoFE_f_139,autoFE_f_79,autoFE_f_132,autoFE_f_209,autoFE_f_89,autoFE_f_208,autoFE_f_204,autoFE_f_178,autoFE_f_199,autoFE_f_242,autoFE_f_250,autoFE_f_247,autoFE_f_149,autoFE_f_134,autoFE_f_273,autoFE_f_148,autoFE_f_102,autoFE_f_112,autoFE_f_185,autoFE_f_198,autoFE_f_285,autoFE_f_104,autoFE_f_58,autoFE_f_186,autoFE_f_293,autoFE_f_150,autoFE_f_195,autoFE_f_82,autoFE_f_87,autoFE_f_200,autoFE_f_155,autoFE_f_124,autoFE_f_189,autoFE_f_246,autoFE_f_284,autoFE_f_179,autoFE_f_140,yield
0,0.399367,0.399367,0.408088,12.537955,0.807455,0.162977,31.793936,31.802657,-30.995202,-30.986480,31.394569,31.394569,31.0,0.123441,0.399367,0.251378,12.811749,0.109506,0.109490,0.109506,0

In [ ]:
test_pred[test_pred['fruitset_x_fruitmass'] == 0.1346910955928016]

In [128]:
submission = pd.read_csv('sample_submission.csv')['id']

submission_df = pd.concat([submission, test_pred[TARGET]], axis=1)
submission_df


,id,yield
0,15289,4251.973443
1,15290,6036.019978
2,15291,7239.748855
3,15292,4503.161860
4,15293,4211.327659
...,...,...
10189,25478,5468.394201
10190,25479,5632.550356
10191,25480,6470.073753
10192,25481,4500.995972


In [129]:
submission_df.to_csv('hist_3cv_openfe_mi_manualcorrect_355.8268.csv', index=False)

## Stacking

In [ ]:
meta_model = Ridge()

In [ ]:
%%time

meta_scores = []

for i, (train_idx, meta_idx) in enumerate(k10.split(X_train)):
    print(f'Fold {i + 1}')
    X_train_meta, X_test_meta = X_train.iloc[train_idx], X_train.iloc[meta_idx]
    y_train_meta, y_test_meta = y_train.iloc[train_idx], y_train.iloc[meta_idx]

    print(X_train_meta.shape, X_test_meta.shape, y_train_meta.shape, y_test_meta.shape)
    
    meta_features_fold = np.zeros((X_test_meta.shape[0], len(models)))
    # meta_test_features = np.zeros((y.shape[0], len(models)))
    # meta_targets = np.zeros(y.shape[0])

    for i, model in enumerate(models):
        model_name = model.__class__.__name__ if not hasattr(model, 'name') else model.name
        print(f'Starting {model_name}')
        model_features = sfs_features[model_name]
        # model_features = baseline_features[model_name]

        # Fit model on the selected features
        model.fit(X_train_meta[model_features], y_train_meta)
        preds = model.predict(X_test_meta[model_features])
        meta_features_fold[:, i] = preds

    # Train the meta-model on the predictions from the base models
    meta_model.fit(meta_features_fold, y_test_meta)
    
    # Predict using the meta-model
    final_preds = meta_model.predict(meta_features_fold)
    
    # Calculate r2_score for the current fold
    current_fold_mae = mean_absolute_error(y_test_meta, final_preds)
    meta_scores.append(current_fold_mae)

In [ ]:
# Calculate the average MAE across all folds
average_mae = np.mean(meta_scores)
average_mae

# 350.36268333368736 Baseline Features
# 348.51521661303786 SFS Features

In [ ]:
%%time

# Retrain base models on all data
all_base_model_predictions = []

for model in models:
    model_name = model.__class__.__name__ if not hasattr(model, 'name') else model.name
    print(f'Starting {model_name}')
    model_features = sfs_features[model_name]
    # model_features = baseline_features[model_name]

    model.fit(X_train[model_features], y_train)
    preds = model.predict(X_test[model_features])

    all_base_model_predictions.append(preds.reshape(-1, 1))

# Stack predictions for the meta model
X_new_meta = np.hstack(all_base_model_predictions)

# Use the meta model to make final predictions
final_predictions = meta_model.predict(X_new_meta)

In [ ]:
final_predictions = pd.DataFrame(final_predictions, columns=[TARGET])
final_predictions = pd.concat([X_test, final_predictions], axis=1)
final_predictions

In [ ]:
model_names = []
for model in models:
    model_name = model.__class__.__name__ if not hasattr(model, 'name') else model.name
    model_names.append(model_name)
model_names

print('Ensemble weights')
weights = pd.Series(meta_model.coef_, index=model_names)
print(weights)
print(f'Weights total: {weights.sum()}')
print(f'Intercept: {meta_model.intercept_}', end='\n\n')
print(f"Average Stacking MAE across all folds: {average_mae:.5f}")

In [ ]:
# Get list of all feature columns (excluding TARGET)
feature_columns = df_train.columns[df_train.columns != TARGET]

# Initialize a dictionary to store corrections
corrections = {}

# Iterate over each feature column
for column in feature_columns:
    # Count occurrences of each value in the column
    value_counts = df_train[column].value_counts()

    # Filter values that appear at least twice
    values_to_check = value_counts[value_counts >= 2].index
     
    # Iterate over each value in the column that appears at least twice
    for value in values_to_check:
        # Get rows where column equals value
        rows_with_value = df_train[df_train[column] == value]

        # Check if all rows with this value in column have the same target
        if rows_with_value[TARGET].nunique() == 1:
            target_value = rows_with_value[TARGET].iloc[0]
        
            # Store correction if it matches training pattern
            if (column, value, target_value) not in corrections:
                corrections[(column, value)] = target_value

In [ ]:
corrections

In [ ]:
# Apply corrections to final_predictions
for (column, value), target_value in corrections.items():
    final_predictions.loc[final_predictions[column] == value, TARGET] = target_value

# Print corrected predictions
print("Corrected predictions in final_predictions:")
final_predictions

In [ ]:
final_predictions_df = pd.DataFrame(final_predictions[TARGET], columns=[TARGET])
final_predictions_df.head()

In [ ]:
submission = pd.read_csv('sample_submission.csv')['id']

submission_stack_df = pd.concat([submission, final_predictions_df[TARGET]], axis=1)
submission_stack_df

In [ ]:
submission_stack_df.to_csv(f'stack_3cv_sfs_manualcorrect_{average_mae}.csv', index=False)